# Homework 3 (main)

#### Kaggle team: < >
#### Group: A0148008J


## Importing, Constants, and data loading functions

In [1]:
# import statement and extra libraries used
import numpy as np
import pandas as pd
import os
import csv
import math
import matplotlib.pyplot as plt
import sklearn
import statistics as st
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from datetime import datetime, date
from datetime import timedelta



In [2]:
# Constant used in this homework
DATE_TIME_FORMAT_DEV = "%d/%m/%Y"
DATE_TIME_FORMAT_REAL = "%Y-%m-%d"
DATE_TIME_FORMAT_WEEK = "%Y-W%W-%w"
MIN_BOOLEAN_INDEX_TRAIN = 5
MAX_BOOLEAN_INDEX_TRAIN = 8
RAW_FEATURE_NUMBER_TRAIN = 9
MIN_BOOLEAN_INDEX_TEST = 4
MAX_BOOLEAN_INDEX_TEST = 7
RAW_FEATURE_NUMBER_TEST = 8
STORE_COMPETITION_SINCE_DEFAULT_TIME = datetime(2009, 3, 9, 18, 13, 5)
STORE_NO_PROMOTION_SINCE_CONSTANT_TIME = datetime(2999, 1, 1, 0, 0, 0) # we assume this datetime is big enough
STORE_NO_COMPETITION_SINCE_CONSTANT_TIME = datetime(2999, 1, 1, 0, 0, 0) # we assume this datetime is big enough
STORE_NO_PROMO_INTERVAL_STRING = "No Promotion"

In [11]:
# File path processing
current_pwd = os.getcwd()
#directory_path = os.path.join(current_pwd, "inpublic/homework3")
train_file_path = os.path.join(directory_path, "train_v2.csv")
test_file_path = os.path.join(directory_path, "test_v2.csv")
store_info_path = os.path.join(directory_path, "store.csv")

In [12]:
# data loading and extraction function
def dataLoadExtract(filePath, booleanMin, booleanMax, stateHolidayIndex):
    rawDataMatrix = []
    firstRow = True
    
    with open(filePath, newline='') as csvFile:
        train_raw = csv.reader(csvFile, delimiter=',')
        for row in train_raw:
            if (firstRow):
                rawDataMatrix.append(row)
                firstRow = False
            else:
                currentRow = []
                for i in range(len(row)):
                    if (i == stateHolidayIndex):
                        currentRow.append(row[i])
                    elif booleanMin<=i<=booleanMax:
                        if (row[i] == '0'):
                            currentRow.append(False)
                        else:
                            currentRow.append(True)
                    elif '-' in row[i]:
                        currentRow.append(
                            datetime.strptime(row[i], DATE_TIME_FORMAT_REAL).date())
                    else:
                        currentRow.append(int(row[i]))
                rawDataMatrix.append(currentRow)
    
    headerRaw = rawDataMatrix[0] # a list containing all the headers as string
    dataRaw = np.array(rawDataMatrix[1:]) # a numpy array with raw data
    return headerRaw, dataRaw

In [13]:
def storeLoadExtract(filePath):
    rawDataMatrix = []
    firstRow = True
    
    with open(filePath, newline='') as csvFile:
        train_raw = csv.reader(csvFile, delimiter=',')
        for row in train_raw:
            if (firstRow):
                header = ["Store Index", "Store Type", "Assortment", "Competition distance reciprocal", "Competition Since",
                          "Promotion Since", "Promotion Interval"]
                rawDataMatrix.append(header)
                firstRow = False
            else:
                currentRow = []
                currentRow.append(int(row[0])) # store index
                currentRow.append(row[1]) # store type
                currentRow.append(row[2]) # assortment
                
                if (row[3] == ""): # competition distance reciprocal
                    currentRow.append(0)
                    currentRow.append(STORE_NO_COMPETITION_SINCE_CONSTANT_TIME)
                else:
                    currentRow.append(1.0/int(row[3])) 
                
                    if (row[4] != ""): 
                        date_str = "1/"+row[4]+"/"+row[5]
                        date_object = datetime.strptime(date_str, DATE_TIME_FORMAT_DEV)
                        currentRow.append(date_object) # competition since time
                    else:
                        currentRow.append(STORE_COMPETITION_SINCE_DEFAULT_TIME)
                
                if (row[6] == "0"): # promotion specs
                    currentRow.append(STORE_NO_PROMOTION_SINCE_CONSTANT_TIME)
                    currentRow.append(STORE_NO_PROMO_INTERVAL_STRING)
                else:
                    date_str = row[8]+"-W"+row[7]+"-0"
                    date_object = datetime.strptime(date_str, DATE_TIME_FORMAT_WEEK)
                    currentRow.append(date_object)
                    currentRow.append(row[9])

                rawDataMatrix.append(currentRow)
    
    headerRaw = rawDataMatrix[0] # a list containing all the headers as string
    dataRaw = np.array(rawDataMatrix[1:]) # a numpy array with raw data
    return headerRaw, dataRaw
    

## Data Loading _(Test_v2.csv, train_v2.csv, store.csv)_

In [14]:
# Data loading and extraction
headerRawTrain, dataRawTrain = dataLoadExtract(train_file_path, MIN_BOOLEAN_INDEX_TRAIN, MAX_BOOLEAN_INDEX_TRAIN, 7)
headerRawTest, dataRawTest = dataLoadExtract(test_file_path, MIN_BOOLEAN_INDEX_TEST, MAX_BOOLEAN_INDEX_TEST, 6)
headerRawStore, dataRawStore = storeLoadExtract(store_info_path)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/DongShaocong/Desktop/CS3244/MachineLearning-CS3244/inpublic/homework3/inpublic/homework3/train_v2.csv'

## Data Manipulation and features conversion

In [7]:
def storeInfoConverter (headerStore, dataStore, dataTrain):
    newHeaderStore = headerStore.copy() # header processing
    newHeaderStore += ["Average Sales Without Promotion",
                       "Average Sales With Promotion",
                       "Average Sales",
                       "Variance Sales Without Promotion",
                       "Variance Sales With Promotion",
                       "Variance Sales",
                       "Average SC Ratio Without Promotion",
                       "Average SC Ratio With Promotion",
                       "Average SC Ratio",
                       "Variance SC Ratio Without Promotion",
                       "Variance SC Ratio With Promotion",
                       "Variance SC Ratio",
                       "Median Sales Without Promotion",
                       "Median Sales With Promotion",
                       "Median Sales",
                       "Average Open Ratio"]
    
    resultant_list = []
    for row in dataStore:
        currentRow = list(row.copy())
        
        # TO DO : maybe we should process the original raw categorical data here
        
        currentStore = int(row[0])
        sales_list_without_promotion = []
        sales_list_with_promotion = []
        sc_ratio_list_without_promotion = []
        sc_ratio_list_with_promotion = []
        openDayCount = 0
        entryCount = 0
        
        for salesRow in dataTrain:
            if (int(salesRow[0]) == currentStore): # It's the store we want to analyze in this round
                
                # sales centric
                if (salesRow[3] > 0):
                    sc_ratio = salesRow[3]/salesRow[4]
                    if (salesRow[6] is True):
                        sales_list_with_promotion.append(salesRow[3])
                        sc_ratio_list_with_promotion.append(sc_ratio)
                    else:
                        sales_list_without_promotion.append(salesRow[3])
                        sc_ratio_list_without_promotion.append(sc_ratio)
                
                # open centric
                if (salesRow[5] is True):
                    openDayCount += 1
            entryCount += 1
        
        # data processing and adding
        currentRow.append(st.mean(sales_list_without_promotion))
        currentRow.append(st.mean(sales_list_with_promotion))
        currentRow.append(st.mean(sales_list_with_promotion+sales_list_without_promotion))
        currentRow.append(st.variance(sales_list_without_promotion))
        currentRow.append(st.variance(sales_list_with_promotion))
        currentRow.append(st.variance(sales_list_with_promotion+sales_list_without_promotion))
        currentRow.append(st.mean(sc_ratio_list_without_promotion))
        currentRow.append(st.mean(sc_ratio_list_with_promotion))
        currentRow.append(st.mean(sc_ratio_list_with_promotion+sc_ratio_list_without_promotion))
        currentRow.append(st.variance(sc_ratio_list_without_promotion))
        currentRow.append(st.variance(sc_ratio_list_with_promotion))
        currentRow.append(st.variance(sc_ratio_list_with_promotion+sc_ratio_list_without_promotion))
        currentRow.append(st.median(sales_list_without_promotion))
        currentRow.append(st.median(sales_list_with_promotion))
        currentRow.append(st.median(sales_list_with_promotion+sales_list_without_promotion))
        currentRow.append(float(openDayCount/entryCount))
        
        # Recording of data
        resultant_list.append(currentRow)
    return newHeaderStore, np.array(resultant_list)

In [8]:
# header information processing
headerStore, dataStore = storeInfoConverter(headerRawStore, dataRawStore, dataRawTrain)

In [9]:
def trainDataLabelSplit(dataTrain, headerTrain):
    # This function makes the training data exactly in the same format of testing data read by our functions
    dataTrainNew = np.hstack((dataTrain[:,:3], dataTrain[:,4:])) # remove the sales column
    return dataTrainNew, headerTrain[:3]+headerTrain[4:], dataTrain[:,3] # return data, header and label
    

In [10]:
# train data label split
dataTrain, headerTrain, labelTrain= trainDataLabelSplit(dataRawTrain, headerRawTrain)

In [11]:
def storeInfoPromotionIntervalConverter(string):
    if string == 'Jan,Apr,Jul,Oct':
        return [1,4,7,10]
    elif string == 'Feb,May,Aug,Nov':
        return [2,5,8,11]
    elif string == 'Mar,Jun,Sep,Dec':
        return [3,6,9,12]
    else:
        return []

In [12]:
def storeNaturalJoin(dataStore, data, headerStore, header):
    headerProcessed = ["Day",
                       "Month",
                       "Month In Promotion"]
    newHeader=  header + headerProcessed + headerStore[1:] # no repetitive store index in header    
    resultant_list = []
    
    for row in data:
        currentIndex = row[0]
        currentStoreInfo = dataStore[currentIndex-1,:] # get corresponding store entry with store index to be removed later
        currentDate = row[2] # we will get the current datetime object
        currentDay = currentDate.day # day value (integer)
        currentMonth = currentDate.month # month value (integer)
        monthInPromotion = currentMonth in storeInfoPromotionIntervalConverter(currentStoreInfo[6]) # boolean
        currentRow = list(row.copy()) + [currentDay, currentMonth, monthInPromotion] # new entries 
        currentRow += list(currentStoreInfo[1:])
        
        resultant_list.append(currentRow)
        
    return newHeader, np.array(resultant_list)
    

In [13]:
# clean the data, and make it ready to use
headerTrainClean, dataTrainClean = storeNaturalJoin(dataStore, dataTrain, headerStore, headerTrain)

In [14]:
# clean the testing features and make it ready to use
headerTestClean, dataTestClean = storeNaturalJoin(dataStore, dataRawTest, headerStore, headerRawTest)

In [15]:
def removeZeroInDataTestClean(dataTestClean, openBooleanIndex):
    resultantList = []
    for row in dataTestClean:
        if (row[openBooleanIndex] is True): # the store is opened on that day
            resultantList.append(list(row))
    return np.array(resultantList)

In [16]:
def removeZeroInLabel(label):
    result = []
    for i in label:
        if i != 0:
            result.append(i)
    return np.array(result)

In [17]:
# we find headerTestClean[4] is 'open'
openBooleanIndex = 4
dataTestRealClean = removeZeroInDataTestClean(dataTestClean, openBooleanIndex)
dataTrainRealClean = removeZeroInDataTestClean(dataTrainClean, openBooleanIndex)

In [18]:
# we remove the zero entries in the labels
labelTrainReal = removeZeroInLabel(labelTrain)

## XGBoost method - initial attempt

In [19]:
def singleFeatureOneHotKeyEncoder(feature_column_vector_train, feature_column_vector_test):
    # numerical encoding
    enc = LabelEncoder()
    featureListTrain = list(feature_column_vector_train)
    featureListTest = list(feature_column_vector_test)
    
    # we should fit the one has larger value set
    if (len(set(featureListTrain)) > len(set(featureListTest))):
        enc.fit(featureListTrain) 
    else:
        enc.fit(featureListTest)
    
    labelEncodedFeatureTrain = enc.transform(featureListTrain).reshape(-1, 1)
    labelEncodedFeatureTest = enc.transform(featureListTest).reshape(-1, 1)
    
    # oneHot encoding
    enc = OneHotEncoder()
    enc.fit(labelEncodedFeatureTrain) # use train to fit the data
    
    # return order: train, test
    return enc.transform(labelEncodedFeatureTrain).toarray(), enc.transform(labelEncodedFeatureTest).toarray()
    

In [20]:
def singleDateTimeColumnNumericalTransformer(dateTimeFeatureColumn):
    processed = np.array([(t-datetime(1970,1,1)).total_seconds()/10**10 for t in dateTimeFeatureColumn])
    return processed.reshape(len(processed),)

In [21]:
def singleDateOnlyColumnNumericalTransformer(dateTimeFeatureColumn):
    processed = np.array([(t-date(1970,1,1)).total_seconds()/10**9 for t in dateTimeFeatureColumn])
    return processed.reshape(len(processed),)

In [48]:
def numericalTrainTransformation(headerTrain, dataTrain, headerTest, dataTest):
    # This function prepares from the clean data to all numerical numpy array ready to be feed into DMatrix
    # We will finally safely remove the store index column
    
    # TO DO : process and return the headerNew as well (for both train and test, they should be the same)
    headerNew = [ "DayOfWeek1", "DayOfWeek2", "DayOfWeek3", "DayOfWeek4", 
                  "DayOfWeek5", "DayOfWeek6", "DayOfWeek7",
                  "Number of Customers", "Open -dummy -True by default",
                  "PromoBoolean1", "PromoBoolean2", "StateHoliday1",
                  "StateHoliday2" ,"StateHoliday3",
                  "SchoolHolidayBoolean", "SchoolHolidayBoolean",
                  "Day", "Month", "Month In Promotion 1", "Month In Promotion 2",
                  "Store type 1", "store type 2", "store type 3", "store type 4",
                  "assortmentType1", "assortmentType2", "assortmentType3",
                  "Competition distance reciprocal", "competitionSinceDate",
                  "Promotion Since Date", "promotionInterval1", "promotionInterval2",
                  "promotionInterval3", "promotionInterval4",
                    'Average Sales Without Promotion',
                 'Average Sales With Promotion',
                 'Average Sales',
                 'Variance Sales Without Promotion',
                 'Variance Sales With Promotion',
                 'Variance Sales',
                 'Average SC Ratio Without Promotion',
                 'Average SC Ratio With Promotion',
                 'Average SC Ratio',
                 'Variance SC Ratio Without Promotion',
                 'Variance SC Ratio With Promotion',
                 'Variance SC Ratio',
                 'Median Sales Without Promotion',
                 'Median Sales With Promotion',
                 'Median Sales',
                 'Average Open Ratio']
    
    # This function should get the training features dict for oneHotEncoded features (key -> number of types)
    
    
    # Categorical features oneHotKey encoding column
    
    
    dayOfWeekColumnTrain, dayOfWeekColumnTest = singleFeatureOneHotKeyEncoder(dataTrain[:,1], dataTest[:,1])
    
    # the date shouldn't affect the prediction
    dateDatetimeColumnTrain = singleDateOnlyColumnNumericalTransformer(dataTrain[:,2])
    dateDatetimeColumnTest = singleDateOnlyColumnNumericalTransformer(dataTest[:,2])
    
    openBooleanTrain, openBooleanTest = singleFeatureOneHotKeyEncoder(dataTrain[:,4], dataTest[:,4]) 
    promoBooleanTrain, promoBooleanTest = singleFeatureOneHotKeyEncoder(dataTrain[:,5], dataTest[:,5]) 
    stateHolidayBooleanTrain, stateHolidayBooleanTest = singleFeatureOneHotKeyEncoder(dataTrain[:,6], dataTest[:,6])
    schoolHolidayBooleanTrain, schoolHolidayBooleanTest = singleFeatureOneHotKeyEncoder(dataTrain[:,7], dataTest[:,7])
    monthInPromotionBooleanTrain, monthInPromotionBooleanTest = singleFeatureOneHotKeyEncoder(dataTrain[:,10], dataTest[:,10])
    storeTypeColumnTrain, storeTypeColumnTest = singleFeatureOneHotKeyEncoder(dataTrain[:,11], dataTest[:,11])
    assortmentTypeColumnTrain, assortmentTypeColumnTest = singleFeatureOneHotKeyEncoder(dataTrain[:,12], dataTest[:,12])
    
    competitionSinceDatetimeColumnTrain = singleDateTimeColumnNumericalTransformer(dataTrain[:,14])
    competitionSinceDatetimeColumnTest = singleDateTimeColumnNumericalTransformer(dataTest[:,14])
    promotionSinceDatetimeColumnTrain = singleDateTimeColumnNumericalTransformer(dataTrain[:,15])
    promotionSinceDatetimeColumnTest = singleDateTimeColumnNumericalTransformer(dataTest[:,15])
    
    promotionIntervalColumnTrain, promotionIntervalColumnTest = singleFeatureOneHotKeyEncoder(dataTrain[:,16], dataTest[:,16])
    
    # use numpy.column_stack to accomplish column and matrix side by side stacking
    resultantArrayTrain = np.column_stack((dayOfWeekColumnTrain,                                      
                                      dataTrain[:,3],
                                      openBooleanTrain,
                                      promoBooleanTrain,
                                      stateHolidayBooleanTrain,
                                      schoolHolidayBooleanTrain,
                                      dataTrain[:,8:10],
                                      monthInPromotionBooleanTrain,
                                      storeTypeColumnTrain,
                                      assortmentTypeColumnTrain,
                                      dataTrain[:,13],
                                      competitionSinceDatetimeColumnTrain,
                                      promotionSinceDatetimeColumnTrain,
                                      promotionIntervalColumnTrain,
                                      dataTrain[:,17:]))
    
    resultantArrayTest = np.column_stack((dayOfWeekColumnTest,
                                      dataTest[:,3],
                                      openBooleanTest,
                                      promoBooleanTest,
                                      stateHolidayBooleanTest,
                                      schoolHolidayBooleanTest,
                                      dataTest[:,8:10],
                                      monthInPromotionBooleanTest,
                                      storeTypeColumnTest,
                                      assortmentTypeColumnTest,
                                      dataTest[:,13],
                                      competitionSinceDatetimeColumnTest,
                                      promotionSinceDatetimeColumnTest,
                                      promotionIntervalColumnTest,
                                      dataTest[:,17:]))
    
    return resultantArrayTrain, resultantArrayTest, headerNew # order: train, test, header

In [23]:
# DataTrainClean categorical feature pre processing
dataTrainNumerical, dataTestNumerical, headerWhole = \
    numericalTrainTransformation(headerTrainClean, dataTrainRealClean, 
                                 headerTestClean, dataTestRealClean)

In [471]:
# Load data and label into xgb.DMatrix
dTrain = xgb.DMatrix(dataTrainNumerical, label=labelTrainReal)
dTest = xgb.DMatrix(dataTestNumerical)
dTrainWithoutLabel = xgb.DMatrix(dataTrainNumerical)

In [117]:
def restoreZeroEntryInPrediction(prediction, dataTestCleanWithZero, openBooleanIndex):
    resultantList= []
    predictionIndex = 0
    for i in range(dataTestCleanWithZero.shape[0]):
        if (dataTestCleanWithZero[i][openBooleanIndex] is True): # the store is open, put our prediction inside
            resultantList.append(prediction[predictionIndex])
            predictionIndex += 1 # update prediction index to the next prediction point
        else:
            resultantList.append(0) # the store is closed, append zero and do not update the index
    return np.array(resultantList).reshape(len(resultantList),)

In [120]:
def writeToFile(numpyArray, filePath):
    with open(filePath, 'w') as csvFile:
        prediction_writer = csv.writer(csvFile, delimiter=',',
                                quotechar='|', quoting=csv.QUOTE_MINIMAL)
        prediction_writer.writerow(["\"Id\"", "\"Sales\""])
        for i in range(numpyArray.shape[0]):
            prediction_writer.writerow([i+1, numpyArray[i]])

#### how we write to file:
* filePath = os.path.join(directory_path, "prediction_3.csv")
* writeToFile(prediction, filePath)

## Second Attempt

#### Actions:
* In this attempt, we try with customized objective functions and evaluation function

In [38]:
# self-defined objective function -> to feed in xgboost, return numpy array
def objectiveFunction(prediction, dTrain):
    labels = dTrain.get_label()
    scale = labels.max()
    grad = -1/labels + prediction/(labels**2)
    hess = 1/(labels**2)
    return scale*grad, scale*hess

In [39]:
# Root Mean Square Percentage Error (RMSPE) return a floating number
def evalError(prediction, dTrain):    
    labels = dTrain.get_label()
    N = labels.shape[0]
    error = 0
    for i in range (N):
        t = labels[i]
        p = prediction[i]
        error += ((p-t)/t)**2
    return 'error', float((error/N)**(1/2))

In [40]:
class RmspeObjective:

    hessian = None

    def __call__(self, predicted, target):
        target = target.get_label()
        # I suspect this is necessary since XGBoost is using 32 bit floats
        # and I'm getting some sort of under/overflow, but that's just a guess
        if self.hessian is None:
            scale = target.max()
            valid = (target > 0)
            self.hessian = np.where(valid, 1.0 / (target / scale)**2, 0)  
        grad = (predicted - target) * self.hessian
        # I suspect (from experiment not from actually reading the relevant paper)
        # that what is important is the ratio of grad to hess.  That's why (I think)
        # I can get away with returning these values, which should be divided by
        # scale**2
        return grad, self.hessian

In [41]:
def errorScore(dTrainWithoutLabel, booster, label):
    prediction = booster.predict(dTrainWithoutLabel)
    N = label.shape[0]
    error = 0
    for i in range (N):
        t = label[i]
        p = prediction[i]
        error += ((p-t)/t)**2
    return float((error/N)**(1/2))
    

## Validation - one round

In [42]:
params = {      'booster'         :'gbtree',
                'objective'       :'reg:linear',
                'nround'          : 3000,
                'colsample_bytree': 0.4,
                }

In [66]:
def trainXGBoost(trainingSet, label, validationSet, validationLabel, testingData, para):
    dTrain = xgb.DMatrix(trainingSet, label=label)
    dTest = xgb.DMatrix(testingData)
    dValidation = xgb.DMatrix(validationSet)
    dTrainWithoutLabel = xgb.DMatrix(trainingSet)
    bst = xgb.train(params=para, dtrain=dTrain, obj = RmspeObjective())
    prediction = bst.predict(dTest)
    error = errorScore(dValidation, bst, validationLabel)
    trainError = errorScore(dTrainWithoutLabel, bst, label)
    print("validation error" + str(error))
    print("training error" + str(trainError))
    print(prediction)
    return prediction, bst, error, trainError

In [72]:
# use the first 6000 rows of the training data as validation set
trainingSet = dataTrainNumerical[9114:,:]
validationSet = dataTrainNumerical[:9114,:]
label = labelTrainReal[9114:]
validationLabel = labelTrainReal[:9114]
prediction , bst, validationError, trainingError= \
    trainXGBoost(trainingSet, label, validationSet, 
                 validationLabel, dataTestNumerical,params)


validation error0.10588355152661916
training error0.0943691597049144
[  4975.296875     5189.609375     7529.80810547 ...,   8129.765625
  20750.515625     6726.33300781]


## Cross validation -> method for time series data

In [80]:
def crossValidationTimeSeries(dataTrainNumerical, label, kFold, dataTestNumerical, para):
    N = dataTrainNumerical.shape[0]
    resultant_validation_error_list = []
    resultant_training_error_list = []
    k = int(math.floor(N/(kFold+1)))
    # print(k)
    
    for multiplier in range(kFold, 0, -1): # the training data we have is in the reverse of time
        
        trainingSet = dataTrainNumerical[k*multiplier:,:]
        validationSet = dataTrainNumerical[k*(multiplier-1):(k*multiplier), :]
        trainingLabel = label[k*multiplier:]
        validationLabel = label[k*(multiplier-1):(k*multiplier)]
        
        prediction , bst, validationError, trainingError= \
            trainXGBoost(trainingSet, trainingLabel, validationSet, 
                         validationLabel, dataTestNumerical, para)
        
        resultant_validation_error_list.append(validationError)
        resultant_training_error_list.append(trainingError)
    
    print(float(sum(resultant_validation_error_list)/len(resultant_validation_error_list))) #average validation error
    return resultant_validation_error_list, resultant_training_error_list

In [81]:
# the format of using our validation function is:
validationError, trainingError = \
    crossValidationTimeSeries(dataTrainNumerical, labelTrainReal, 5, dataTestNumerical, params)

validation error0.19940895403592448
training error0.07099890731883694
[  6279.11669922   6625.29199219   9298.49121094 ...,   8210.61621094
  25093.01757812   7573.85888672]
validation error0.22758069271060466
training error0.07677104624116776
[  5705.67285156   5945.33251953   8395.49414062 ...,   7943.45556641
  23959.03515625   7265.11572266]
validation error0.15874419934571501
training error0.08910849661222453
[  5066.10205078   5930.06396484   7551.01318359 ...,   8512.94824219
  24706.85351562   7238.41308594]
validation error0.17785212265774408
training error0.094826196747337
[  4825.12548828   5413.35595703   7734.078125   ...,   7989.33056641
  23360.56835938   7162.94873047]
validation error0.10588355152661916
training error0.0943691597049144
[  4975.296875     5189.609375     7529.80810547 ...,   8129.765625
  20750.515625     6726.33300781]
0.17389390405532149
